In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0'

In [3]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [4]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [5]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [7]:
X.shape

(10000, 10)

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct =ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder="passthrough")
X = ct.fit_transform(X)

In [9]:
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [10]:
X.shape

(10000, 12)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2)

In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [15]:
ann.compile(optimizer='adam' , loss='binary_crossentropy' , metrics=['accuracy'])

In [16]:
ann.fit(X_train, y_train, batch_size=32, epochs=25)

Train on 8000 samples
Epoch 1/25
8000/8000 [==============================] - 1s 112us/sample - loss: 0.5340 - accuracy: 0.7962
Epoch 2/25
8000/8000 [==============================] - ETA: 0s - loss: 0.4795 - accuracy: 0.79 - 0s 30us/sample - loss: 0.4815 - accuracy: 0.7962
Epoch 3/25
8000/8000 [==============================] - ETA: 0s - loss: 0.4616 - accuracy: 0.79 - 0s 30us/sample - loss: 0.4588 - accuracy: 0.7962
Epoch 4/25
8000/8000 [==============================] - 0s 30us/sample - loss: 0.4447 - accuracy: 0.7962
Epoch 5/25
8000/8000 [==============================] - 0s 29us/sample - loss: 0.4359 - accuracy: 0.7962
Epoch 6/25
8000/8000 [==============================] - 0s 28us/sample - loss: 0.4302 - accuracy: 0.7962
Epoch 7/25
8000/8000 [==============================] - 0s 29us/sample - loss: 0.4257 - accuracy: 0.8005
Epoch 8/25
8000/8000 [==============================] - 0s 30us/sample - loss: 0.4221 - accuracy: 0.8161
Epoch 9/25
8000/8000 [==============================]

In [17]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

[[False]]


In [18]:
y_pred = ann.predict(X_test)
for i in range(y_pred.shape[0]):
    for j in range(y_pred.shape[1]):
        if(y_pred[i][j]>0.5):
            y_pred[i][j]=1
        else:
            y_pred[i][j]=0
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1545,   48],
       [ 242,  165]], dtype=int64)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.855